In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup


In [117]:
starbucks_response = requests.get("https://worldpopulationreview.com/state-rankings/starbucks-stores-by-state")

In [118]:
from bs4 import BeautifulSoup
starbs_soup = BeautifulSoup(starbucks_response.content, "html.parser")

In [119]:
star_table=(starbs_soup.find_all("table"))[0]
star_table

<table class="wpr-table"><thead class="table-head bg-wpr-table_header_bg relative z-40 px-0 py-0 text-xs uppercase tracking-wider"><tr><th class="datatable-th bg-wpr-table_header_bg text-wpr-table_header border-wpr-table_border sticky top-0 border-b bg-clip-padding px-3 py-2 text-left align-bottom leading-4 md:px-4 pin left-0 z-50 border-l-0"><div class="flex flex-col items-stretch justify-start"><div class="flex flex-row items-center cursor-pointer select-none">State</div></div></th><th class="datatable-th bg-wpr-table_header_bg text-wpr-table_header border-wpr-table_border sticky top-0 border-b bg-clip-padding px-3 py-2 text-left align-bottom leading-4 md:px-4 z-40"><div class="flex flex-col items-stretch justify-start"><div class="flex flex-row items-center cursor-pointer select-none">Starbucks Stores 2023<svg class="icon icon-tabler icon-tabler-chevron-down" fill="none" height="14" stroke="currentColor" stroke-linecap="round" stroke-linejoin="round" stroke-width="2" viewbox="0 0 24

In [120]:
row = table.find_all("tr")[1]
row

<tr class="table-row" data-state="false"><th class="text-wpr-table_header sticky left-0 z-50 bg-clip-padding py-1.5 pl-3 text-left align-middle text-sm md:px-4"><a class="text-wpr-link" href="/states/california" to="/states/california">California</a></th><td class="text-wpr-table_text z-40 px-3 py-1.5 text-sm md:px-4">3,080</td><td class="text-wpr-table_text z-40 px-3 py-1.5 text-sm md:px-4">2,959</td><td class="text-wpr-table_text z-40 px-3 py-1.5 text-sm md:px-4">3,117</td></tr>

In [121]:
  state= row.find("th").text
  state

'California'

In [122]:
cells = row.find_all('td')
cells

[<td class="text-wpr-table_text z-40 px-3 py-1.5 text-sm md:px-4">3,080</td>,
 <td class="text-wpr-table_text z-40 px-3 py-1.5 text-sm md:px-4">2,959</td>,
 <td class="text-wpr-table_text z-40 px-3 py-1.5 text-sm md:px-4">3,117</td>]

In [123]:
year2023 = cells[0].text
year2023

'3,080'

In [124]:
starbs= []

# iterate over all rows in the population table
for row in star_table.find_all("tr")[1:]:

  #state
  state= row.find("th").text

#Cells for num of starbucks
  cells = row.find_all('td')
  #Year
  year2023 = cells[0].text
  year2021 = cells[1].text
  year2024 = cells[2].text

  

  starbs.append({
                "State": state,
                "2024": year2024,
                "2023": year2023,
                "2021": year2021,
  })

In [147]:
starbs = pd.DataFrame(starbs)

In [148]:
#melt
starbs_melt= starbs.melt(id_vars=["State"], var_name='Year', value_name="Stores")

In [149]:
starbs_melt["Chain"] = "Starbucks"
starbs_melt.head()

,State,Year,Stores,Chain
0,California,2024,"3,117",Starbucks
1,Texas,2024,"1,409",Starbucks
2,Florida,2024,892,Starbucks
3,Washington,2024,736,Starbucks
4,New York,2024,715,Starbucks


In [112]:
dunkin_response = requests.get("https://worldpopulationreview.com/state-rankings/dunkin-donuts-by-state")

In [113]:
from bs4 import BeautifulSoup
dunks_soup = BeautifulSoup(dunkin_response.content, "html.parser")

In [114]:
dunkin_table=(dunks_soup.find_all("table"))[0]
#dunkin_table

In [115]:
dunky= []

# iterate over all rows in the population table
for row in dunkin_table.find_all("tr")[1:]:

  #state
  state= row.find("th").text

  #Cells for num of dunkin
  cells = row.find_all('td')
  #Year
  year2024 = cells[0].text
  year2023 = cells[1].text
 

  

  dunky.append({
                "State": state,
                "2024": year2024,
                "2023": year2023
               
  })

In [140]:
dunky =pd.DataFrame(dunky)

In [143]:
#melt
dunky_melt= dunky.melt(id_vars=["State"], var_name='Year', value_name="Stores")

In [146]:
dunky_melt["Chain"] = "Dunkin"
dunky_melt.head()

,State,Year,Stores,Chain
0,New York,2024,"1,431",Dunkin
1,Massachusetts,2024,"1,042",Dunkin
2,Florida,2024,909,Dunkin
3,New Jersey,2024,872,Dunkin
4,Illinois,2024,711,Dunkin


In [150]:
starbs_dunky= pd.concat([starbs_melt, dunky_melt])
starbs_dunky

,State,Year,Stores,Chain
0,California,2024,"3,117",Starbucks
1,Texas,2024,"1,409",Starbucks
2,Florida,2024,892,Starbucks
3,Washington,2024,736,Starbucks
4,New York,2024,715,Starbucks
...,...,...,...,...
148,Montana,Chain,Dunkin,Dunkin
149,North Dakota,Chain,Dunkin,Dunkin
150,Oregon,Chain,Dunkin,Dunkin
151,South Dakota,Chain,Dunkin,Dunkin


In [12]:
#combine
merge_df = pd.merge(starbs, dunky, how="inner", on='State')

In [13]:
merge_df

,state,Starbucks Stores,Dunkin Stores
0,California,"3,080",143
1,Texas,"1,346",226
2,Florida,844,909
3,Washington,741,0
4,New York,692,"1,431"
5,Illinois,677,711
6,Arizona,548,110
7,Colorado,495,48
8,Ohio,491,261
9,Virginia,489,244


In [14]:
#wiki table
wiki_response = requests.get("https://simple.wikipedia.org/wiki/List_of_U.S._states_by_population")

In [15]:
from bs4 import BeautifulSoup
wiki_soup = BeautifulSoup(wiki_response.content, "html.parser")

In [16]:
wiki_table=(wiki_soup.find_all("table"))[0]
#wiki_table


In [17]:
row =wiki_table.find_all("tr")[1]
#row

In [18]:
cell= row.find_all('td')
cell[2].get_text(strip=True)

'California'

In [20]:
wiki = []

# iterate over all rows in the population table
for row in wiki_table.find_all("tr")[1:]:


    # Get all the cells (<td>) in the row.
    #state
    cells = row.find_all("td")
    state=cells[2].get_text(strip=True)


 #find population
    pop = cells[3].get_text(strip=True)


    # Append this data.
    wiki.append({
        "state": state,
        "Population": pop,
})

In [21]:
wiki=pd.DataFrame(wiki)

In [22]:
final_merged = pd.merge(wiki, merge_df, how="inner", on='state')


In [23]:
starbucks_stock= 96
dunkin_stock = 106
final_merged["starbucks_stock"]=starbucks_stock
final_merged["dunkin_stock"]=dunkin_stock

In [24]:
final_merged

,state,Population,Starbucks Stores,Dunkin Stores,starbucks_stock,dunkin_stock
0,California,"39,538,223","3,080",143,96,106
1,Texas,"30,145,505","1,346",226,96,106
2,Florida,"21,538,187",844,909,96,106
3,New York,"20,201,249",692,"1,431",96,106
4,Pennsylvania,"13,002,700",357,650,96,106
5,Illinois,"12,812,508",677,711,96,106
6,Ohio,"11,799,448",491,261,96,106
7,Georgia,"10,711,908",326,287,96,106
8,North Carolina,"10,439,388",338,224,96,106
9,Michigan,"10,077,331",283,112,96,106


In [25]:
#map
states_regions = pd.DataFrame({
    'state': [
        'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado',
        'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
        'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
        'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
        'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
        'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
        'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
        'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
        'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
        'West Virginia', 'Wisconsin', 'Wyoming'
    ],
    'region': [
        'South', 'West', 'West', 'South', 'West', 'West', 'Northeast', 
        'South', 'South', 'South', 'West', 'West', 'Midwest', 'Midwest', 
        'Midwest', 'Midwest', 'South', 'South', 'Northeast', 'South', 
        'Northeast', 'Midwest', 'Midwest', 'South', 'Midwest', 'West', 
        'Midwest', 'West', 'Northeast', 'Northeast', 'West', 'Northeast',
        'South', 'Midwest', 'Midwest', 'South', 'West', 'Northeast', 
        'Northeast', 'South', 'Midwest', 'South', 'South', 'West', 
        'Northeast', 'South', 'West', 'South', 'Midwest', 'West'
    ]
})
states_regions.head()

,state,region
0,Alabama,South
1,Alaska,West
2,Arizona,West
3,Arkansas,South
4,California,West


In [26]:
final_merged = pd.merge(final_merged, states_regions, on='state')

In [27]:
sort = final_merged.sort_values(by='Starbucks Stores', ascending=False)
sort

,state,Population,Starbucks Stores,Dunkin Stores,starbucks_stock,dunkin_stock,region
39,Hawaii,"1,455,271",99,6,96,106,West
34,Kansas,"2,937,880",94,26,96,106,Midwest
30,Iowa,"3,190,369",89,36,96,106,Midwest
23,Alabama,"5,024,279",85,69,96,106,South
2,Florida,"21,538,187",844,909,96,106,South
24,Louisiana,"4,657,757",84,15,96,106,South
48,Vermont,"643,077",8,50,96,106,Northeast
27,Oklahoma,"3,959,353",79,23,96,106,South
35,New Mexico,"2,117,522",76,19,96,106,West
12,Washington,"7,705,281",741,0,96,106,West


In [28]:
final_merged['Starbucks Stores'] = final_merged['Starbucks Stores'].str.replace(',', '').astype(int)

In [29]:
final_merged['Dunkin Stores'] = final_merged['Dunkin Stores'].str.replace(',', '').astype(int)

In [32]:
final_merged.head()

,state,Population,Starbucks Stores,Dunkin Stores,starbucks_stock,dunkin_stock,region
0,California,"39,538,223",3080,143,96,106,West
1,Texas,"30,145,505",1346,226,96,106,South
2,Florida,"21,538,187",844,909,96,106,South
3,New York,"20,201,249",692,1431,96,106,Northeast
4,Pennsylvania,"13,002,700",357,650,96,106,Northeast


In [33]:
sort= final_merged.sort_values(by='Starbucks Stores', ascending=False)
sort

,state,Population,Starbucks Stores,Dunkin Stores,starbucks_stock,dunkin_stock,region
0,California,"39,538,223",3080,143,96,106,West
1,Texas,"30,145,505",1346,226,96,106,South
2,Florida,"21,538,187",844,909,96,106,South
12,Washington,"7,705,281",741,0,96,106,West
3,New York,"20,201,249",692,1431,96,106,Northeast
5,Illinois,"12,812,508",677,711,96,106,Midwest
13,Arizona,"7,151,502",548,110,96,106,West
20,Colorado,"5,773,714",495,48,96,106,West
6,Ohio,"11,799,448",491,261,96,106,Midwest
11,Virginia,"8,631,393",489,244,96,106,South


In [31]:
sort_region= final_merged.sort_values(by='region', ascending=False)
sort_region

,state,Population,Starbucks Stores,Dunkin Stores,starbucks_stock,dunkin_stock,region
0,California,"39,538,223",3080,143,96,106,West
12,Washington,"7,705,281",741,0,96,106,West
47,Alaska,"733,391",49,0,96,106,West
43,Montana,"1,084,225",32,0,96,106,West
39,Hawaii,"1,455,271",99,6,96,106,West
37,Idaho,"1,839,106",67,0,96,106,West
35,New Mexico,"2,117,522",76,19,96,106,West
31,Nevada,"3,104,614",253,42,96,106,West
29,Utah,"3,271,616",101,2,96,106,West
26,Oregon,"4,237,256",359,0,96,106,West


In [32]:
final_merged['proportion_of_Starbucks'] = final_merged['Starbucks Stores'] / final_merged['Starbucks Stores'].sum()

In [33]:
final_merged['proportion_of_Dunkin'] = final_merged['Dunkin Stores'] / final_merged['Dunkin Stores'].sum()

In [34]:
final_merged

,state,Population,Starbucks Stores,Dunkin Stores,starbucks_stock,dunkin_stock,region,proportion_of_Starbucks,proportion_of_Dunkin
0,California,"39,538,223",3080,143,96,106,West,0.210196,0.014823
1,Texas,"30,145,505",1346,226,96,106,South,0.091858,0.023427
2,Florida,"21,538,187",844,909,96,106,South,0.057599,0.094226
3,New York,"20,201,249",692,1431,96,106,Northeast,0.047226,0.148336
4,Pennsylvania,"13,002,700",357,650,96,106,Northeast,0.024364,0.067378
5,Illinois,"12,812,508",677,711,96,106,Midwest,0.046202,0.073702
6,Ohio,"11,799,448",491,261,96,106,Midwest,0.033508,0.027055
7,Georgia,"10,711,908",326,287,96,106,South,0.022248,0.029750
8,North Carolina,"10,439,388",338,224,96,106,South,0.023067,0.023220
9,Michigan,"10,077,331",283,112,96,106,Midwest,0.019313,0.011610


There are more overall Starbucks stores than Dunkin. However in teh Northeast it looks like there are more Dunkin stopres relativly. 

In [1]:
def webscrape(url):
    
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    table=(soup.find_all("table"))[0]

    df= []

# iterate over all rows in the population table
    for row in table.find_all("tr")[1:]:

        #state
        state= row.find("th").text

        #number of stores
        num=row.find("td").text

        df.append({
                "State": state,
                "Stores": num,
    })
    df =pd.DataFrame(df)
    return df
    

    

In [36]:
webscrape("https://worldpopulationreview.com/state-rankings/walmart-stores-by-state")

,State,Stores
0,Texas,517
1,Florida,341
2,California,280
3,North Carolina,192
4,Georgia,189
5,Illinois,161
6,Ohio,146
7,Missouri,137
8,Tennessee,137
9,Pennsylvania,134
